# Decorators
- Functions, classes, and methods can be 'decorated'
- Will only show how to decorate functions - others are fairly complex
- Similar to 'annotations and aspect' programming in java
- Good for 'cross cutting' concerns, like security, metering, billing. 
- Surprising what can be done with decorators

# Callables
- a 'callable' is something that can be 'called', applied to arguments
    - have seen functions and lambdas
    - objects can also be callables, by defining the ```__call__``` method

In [1]:
import math

class Co:
    # args applied to object will call this
    def __call__(self, x):
        return(math.sin(x))


# make a Co object
c = Co()

# can call object like a function
[math.sin(.5), c(.5)]
    

[0.479425538604203, 0.479425538604203]

In [2]:
# good old recursive factorial, 
# with a print debug statement added

def fact(n):
    print('inside fact({})'.format(n))
    if n == 0:
        return(1)
    else:
        return(n * fact(n-1))

fact(4)


inside fact(4)
inside fact(3)
inside fact(2)
inside fact(1)
inside fact(0)


24

# to decorate a function, define a callable class

In [14]:
class traceindent:
    def __init__(self, func):
        # func is the original function
        # defined below @traceident line
        self.func = func
        self.level = 0

    # when func is called - this method
    # is called, not the original func
    def __call__(self, *pos, **kw):
        self.level += 1
        indent = ['.'] * self.level
        indent = ''.join(indent)
        if len(pos) == 1:
            printpos = '({})'.format(pos[0])
        print("{}Entering {}{}".format(indent, \
                        self.func.__name__, printpos))
        # calling the traced function
        val = self.func(*pos, **kw)
        print('{}Exiting {}{}=>{}'.format(indent, \
                    self.func.__name__, printpos, val))
        self.level -= 1
        return(val)


In [15]:
# removed the print statement from fact
# 'decorate' the fact function with a traceindent

@traceindent
def fact(n):
    if n == 0:
        return(1)
    else:
        return(n * fact(n-1))

In [16]:
fact(4)

.Entering fact(4)
..Entering fact(3)
...Entering fact(2)
....Entering fact(1)
.....Entering fact(0)
.....Exiting fact(0)=>1
....Exiting fact(1)=>1
...Exiting fact(2)=>2
..Exiting fact(3)=>6
.Exiting fact(4)=>24


24

In [17]:
# 'fact' refers to an object instance of 'traceindent',
# the original function object created by 'def fact' is 
# referred by fact.func

[fact, type(fact), fact.func, type(fact.func)]

 __main__.traceindent,
 <function __main__.fact>,
 function]

# functools module
- has some decorators
- [doc](https://docs.python.org/3.5/library/functools.html)


In [18]:
# here only need to define 
# __eq__ and __lt__
# the decorator defines __le__, __ge__, __le__ 

from functools import total_ordering

@total_ordering
class Student:
    def __init__(self, first, last):
        self.first = first
        self.last = last
    def __eq__(self, other):
        # instead of checking first and last names 
        # separately, make tuples 
        # and check those once
        # is there a disadvantage to this technique?
        s = (self.last.lower(), self.first.lower())
        o = (other.last.lower(), other.first.lower())
        return s == o
    def __lt__(self, other):
        s = (self.last.lower(), self.first.lower())
        o = (other.last.lower(), other.first.lower())        
        return s < o

In [19]:
s1 = Student('jack', 'stead')
s2 = Student('larry', 'stead')

# only the first two operators 
# were explicitly defined above
[s1 == s2, s1 < s2, s1 > s2, s1 <= s2, s1 >= s2]

[False, True, False, True, False]

# dynamic programming/memoization
- avoid redoing computations by cacheing results

In [20]:
# f[n] = f[n-1] + f[n-2]
# doubly recursive
# many redundant calls...

def fibonacci(n):
   "Return the nth fibonacci number."
   print('in fib', n)
   if n in (0,1):
      return n
   return fibonacci(n-1) + fibonacci(n-2)

fibonacci(7)

in fib 7
in fib 6
in fib 5
in fib 4
in fib 3
in fib 2
in fib 1
in fib 0
in fib 1
in fib 2
in fib 1
in fib 0
in fib 3
in fib 2
in fib 1
in fib 0
in fib 1
in fib 4
in fib 3
in fib 2
in fib 1
in fib 0
in fib 1
in fib 2
in fib 1
in fib 0
in fib 5
in fib 4
in fib 3
in fib 2
in fib 1
in fib 0
in fib 1
in fib 2
in fib 1
in fib 0
in fib 3
in fib 2
in fib 1
in fib 0
in fib 1


13

In [21]:
import collections
import functools

class memoized(object):
   '''Decorator. Caches a function's return 
   value each time it is called.
   If called later with the same arguments, 
   the cached value is returned
   (not reevaluated).
   '''
   def __init__(self, func):
      self.func = func
      self.cache = {}
        
   def __call__(self, *args):
      if args in self.cache:
         # found previous computation in cache
         return self.cache[args]
      else:
         # add this computation to cache
         value = self.func(*args)
         self.cache[args] = value
         return value
    
   def __repr__(self):
      '''Return the function's docstring.'''
      return self.func.__doc__

@memoized
def fibonaccim(n):
   '''Return the nth fibonacci number.'''
   print('in fib', n)
   if n in (0, 1):
      return n
   return fibonaccim(n-1) + fibonaccim(n-2)

In [22]:
# now no redundant calls

fibonaccim(8)

in fib 8
in fib 7
in fib 6
in fib 5
in fib 4
in fib 3
in fib 2
in fib 1
in fib 0


21

In [23]:
# functools has a better memo decorator

import functools

# maxsize=an int will limit the size of the cache

@functools.lru_cache(maxsize=None)
def fiblru(n):
   "Return the nth fibonacci number."
   print('in fib', n)
   if n in (0, 1):
      return n
   return fiblru(n-1) + fiblru(n-2)


In [24]:
fiblru(8)

in fib 8
in fib 7
in fib 6
in fib 5
in fib 4
in fib 3
in fib 2
in fib 1
in fib 0


21

In [25]:
# info about the cache

fiblru.cache_info()

CacheInfo(hits=6, misses=9, maxsize=None, currsize=9)

In [26]:
# can clear the cache

fiblru.cache_clear()

In [27]:
fiblru.cache_info()

CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)

In [28]:
# oops - can't use a list as a dict key

@functools.lru_cache(maxsize=None)
def cnt(lst):
    return len(lst)

cnt([3,3,4])

TypeError: unhashable type: 'list'

# begins module
- concise alternative to 'argparse'
- uses decorators and function annotations
- name of module is 'begins', but you import 'begin'
- if you want to try it, you have to install it:

pip install begins

# script source 

#!/usr/bin/env python

import begin

```
@begin.start
def run(parg1, parg2, name: 'cmd line doc from function annotation' = 'lstead', enable=True):
    print('run got: parg1={} parg2={} name={} enable={}'\
    	       .format(parg1, parg2, name, enable))

```

# sample output

```
cmdlinebegins -h
usage: cmdlinebegins [-h] [--name NAME] [--enable] [--no-enable] PARG1 PARG2

positional arguments:
  PARG1
  PARG2

optional arguments:
  -h, --help            show this help message and exit
  --name NAME, -n NAME  cmd line doc from function annotation (default: lstead)
  --enable
  --no-enable           (default: True)

cmdlinebegins foo bar -n asdf --no-enable
run got: parg1=foo parg2=bar name=asdf enable=False

```

# [Standard Library of Decorators](https://wiki.python.org/moin/PythonDecoratorLibrary)
- some useful things